# Quantifiers Notebook

This notebook is used to run quantifiers to chimera states and to analyze the data.

In [24]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

$$Z_k = | \frac{1}{2\delta} \sum_{|j-k| \le \delta} e^{i\theta_k}|$$

In [5]:
def param_local_order(spatial_pattern, n_neighbors):
    spatial_pattern = np.array(spatial_pattern)
    n = len(spatial_pattern)
    n_neighbors = int(n_neighbors/2)
    real = np.zeros_like(spatial_pattern, dtype='g')
    complx = np.zeros_like(spatial_pattern, dtype='g')
    Zk = np.zeros_like(spatial_pattern, dtype='g')

    for i in range(n):
        for k in range(-n_neighbors, n_neighbors+1):
            if i!=(k+i): # not autoconection
                real[i%n] += np.cos(2*np.pi*spatial_pattern[(k+i)%n]) # real part
                complx[i%n] += np.sin(2*np.pi*spatial_pattern[(k+i)%n]) # img part       
        Zk[i%n] += real[(i)%n]*real[(i)%n] + complx[(i)%n]*complx[(i)%n] # 
        Zk[i%n] = abs((1.0/(2*n_neighbors))*np.sqrt(Zk[i%n])) # len fasor.

    return Zk

In [13]:
def degree_inconsistency(param_local_order):
        n = len(param_local_order)
        length_coherent = np.zeros_like(param_local_order, dtype='g')
        counter = 1 # counter de grupos (Np)
        key = 0 

        for r in param_local_order:  
            if r > 0.99:
                length_coherent[counter] += 1 # counter of coherent groups
                key = 1
            if r <= 0.99 and key == 1:
                counter += 1 # counter of groups
                key = 0

        Nl = length_coherent.sum()

        degree = Nl/(counter) # Ñ 
        p = degree/n #

        return p, Nl, counter

# Potential

In [ ]:
def plot_fase_space(data, number_cell):
    v = np.array(data['simData']['V_soma'][f'cell_{number_cell}'])[1:]
    dVdt = np.diff(data['simData']['V_soma'][f'cell_{number_cell}'])
    plt.figure()
    plt.title(f'fase space of cell {number_cell}')
    sns.scatterplot(x = v, y=dVdt, s=0.5)

In [ ]:
def plotPotential(time, voltage, stim_current, title):

    f, (ax0, ax1) = plt.subplots(2,1, figsize=(10,3), gridspec_kw = {'height_ratios':[3, 1]})
    sns.lineplot(x = time, y =voltage, color='black', ax=ax0)
    ax1.plot(time,stim_current, 'gray')

    ax0.set_ylabel('Voltage (mV)')
    ax0.spines['right'].set_visible(False)
    ax0.spines['top'].set_visible(False)
    ax0.spines['bottom'].set_visible(False)
    ax0.get_xaxis().set_visible(False)


    ax1.plot([0,0],[0,0.15],'k')
    ax1.text(20,0.125,f'{stim_current.max()}nA',va='center')
    ax1.set_ylabel('I (nA)')
    ax1.set_xlabel('t (ms)')

    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    ax1.get_yaxis().set_visible(False)
    plt.title(title)
    plt.tight_layout()

In [ ]:
def showPotential(label_network, n_cell):
    amp = data[label_network]['simConfig']['IClamp0']['amp']
    dur = data[label_network]['simConfig']['IClamp0']['dur']
    start = data[label_network]['simConfig']['IClamp0']['start']
    recordStep = data[label_network]['simConfig']['recordStep']

    voltage = np.array(list(data[label_network]['simData']['V_soma'][f'cell_{n_cell}']))
    time = np.array(data[label_network]['simData']['t'])
    stim_current = np.array([0 if x < start/recordStep or x > (dur+start)/recordStep else amp for x in range(0, len(time))])
    gex = data[label_network]['simConfig']['gex']
    gex_label = '$g_{ex} = '+ str(gex) +'$'
    plotPotential(time,voltage, stim_current,gex_label)
